In [ ]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Bidirectional, Dense
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

# Define a richer dataset
words = ["MACHINE", "MACHO", "MACRON", "MAGNET", "MATCHES", "MATCHER", "MATCHING", "MATCH", "MACHINERY"]

# Vocabulary
alphabet = sorted(set(''.join(words)))
char_to_int = {c: i+1 for i, c in enumerate(alphabet)}  # start from 1
int_to_char = {i: c for c, i in char_to_int.items()}
vocab_size = len(char_to_int) + 1  # +1 for padding

# Create sequences to predict the last character
sequences = []
labels = []

for word in words:
    input_seq = list(word[:-1])  # remove last char
    label = word[-1]             # target is the last char
    encoded_seq = [char_to_int[c] for c in input_seq]
    sequences.append(encoded_seq)
    labels.append(char_to_int[label])

max_len = max(len(seq) for seq in sequences)
X = pad_sequences(sequences, maxlen=max_len, padding='pre')
y = to_categorical(labels, num_classes=vocab_size)

# Build simple RNN
def build_rnn():
    model = Sequential([
        Embedding(input_dim=vocab_size, output_dim=10),
        SimpleRNN(64),
        Dense(vocab_size, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Build Bidirectional RNN
def build_birnn():
    model = Sequential([
        Embedding(input_dim=vocab_size, output_dim=10),
        Bidirectional(SimpleRNN(64)),
        Dense(vocab_size, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Train models
rnn_model = build_rnn()
rnn_model.fit(X, y, epochs=500, verbose=0)

birnn_model = build_birnn()
birnn_model.fit(X, y, epochs=500, verbose=0)

# Prediction
def predict_last_letter(model, partial_word):
    encoded = [char_to_int[c] for c in partial_word]
    padded = pad_sequences([encoded], maxlen=max_len, padding='pre')
    pred = model.predict(padded, verbose=0)
    return int_to_char[np.argmax(pred)]

# Example: "MACHIN" → Predict "E"
incomplete = list("MACHIN")
rnn_pred = predict_last_letter(rnn_model, incomplete)
birnn_pred = predict_last_letter(birnn_model, incomplete)

print(f"Incomplete word: {''.join(incomplete)}")
print(f"RNN prediction: {rnn_pred}")
print(f"Bidirectional RNN prediction: {birnn_pred}")


Incomplete word: MACHIN
RNN prediction: E
Bidirectional RNN prediction: E


In [ ]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense

# 1. Text Preprocessing
sentences = [
    "The cat sat on the mat",
    "The dog sat on the rug",
    "The bird flew in the sky",
    "The cat jumped over the fence"
]

# Tokenization
tokenizer = Tokenizer()
tokenizer.fit_on_texts(sentences)

# Vocabulary size
vocab_size = len(tokenizer.word_index) + 1

# Create sequences: e.g. "The cat sat" -> predict "on"
input_sequences = []
labels = []

for sentence in sentences:
    tokens = tokenizer.texts_to_sequences([sentence])[0]
    for i in range(1, len(tokens)):
        input_sequences.append(tokens[:i])
        labels.append(tokens[i])

# Pad sequences
max_len = max(len(seq) for seq in input_sequences)
input_sequences = pad_sequences(input_sequences, maxlen=max_len, padding='pre')

# Convert labels to categorical
from tensorflow.keras.utils import to_categorical
labels = to_categorical(labels, num_classes=vocab_size)

# 2. Model Building
model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=10, input_length=max_len),
    SimpleRNN(64),
    Dense(vocab_size, activation='softmax')
])

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# 3. Train the Model
model.fit(input_sequences, labels, epochs=500, verbose=0)

# 4. Prediction
def predict_next_word(text):
    sequence = tokenizer.texts_to_sequences([text])[0]
    padded = pad_sequences([sequence], maxlen=max_len, padding='pre')
    pred = model.predict(padded, verbose=0)
    next_word_id = np.argmax(pred)
    for word, idx in tokenizer.word_index.items():
        if idx == next_word_id:
            return word
    return "?"

# Test
test_input = "The cat sat on"
predicted_word = predict_next_word(test_input)

print(f"Input Sentence: '{test_input}'")
print(f"Predicted Next Word: '{predicted_word}'")

Input Sentence: 'The cat sat on'
Predicted Next Word: 'the'


In [ ]:
import numpy as np
import random
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

# Define full note vocabulary (include variations)
all_notes = ['Sa', 'Re', 'Reb', 'Ga', 'Gab', 'Ma', 'Ma#', 'Pa', 'Dha', 'Dhab', 'Ni', 'Sha']
note_to_int = {note: i for i, note in enumerate(all_notes)}
int_to_note = {i: note for note, i in note_to_int.items()}
vocab_size = len(all_notes)

# Define ragas with simplified sequences
ragas = {
    "Bhairav": ['Sa', 'Reb', 'Ga', 'Ma', 'Pa', 'Dhab', 'Ni', 'Sha'],
    "Bhopali": ['Sa', 'Re', 'Ga', 'Pa', 'Dha', 'Sha'],
    "Bageshree": ['Sa', 'Gab', 'Ma', 'Dha', 'Ni', 'Sha'],
    "Yaman": ['Sa', 'Re', 'Ga', 'Ma#', 'Pa', 'Dha', 'Ni', 'Sha'],
    "Desh": ['Sa', 'Re', 'Ma', 'Pa', 'Ni', 'Sha']
}

# Function to generate sequences
def generate_sequences(raga_notes, n_sequences=500):
    sequences = []
    for _ in range(n_sequences):
        start = random.randint(0, len(raga_notes) - 4)
        seq = raga_notes[start:start+4]
        sequences.append(seq)
    return sequences

# Function to train and generate from a specific raga
def train_and_generate_raga(raga_name, seed_notes, sequence_length=10):
    print(f"\nTraining on Raga: {raga_name}")
    raga_notes = ragas[raga_name]
    sequences = generate_sequences(raga_notes)

    # Preprocess
    X, y = [], []
    for seq in sequences:
        for i in range(1, len(seq)):
            input_seq = [note_to_int[n] for n in seq[:i]]
            label = note_to_int[seq[i]]
            X.append(input_seq)
            y.append(label)

    max_len = max(len(seq) for seq in X)
    X = pad_sequences(X, maxlen=max_len, padding='pre')
    y = to_categorical(y, num_classes=vocab_size)

    # Build RNN
    model = Sequential([
        Embedding(input_dim=vocab_size, output_dim=10, input_length=max_len),
        SimpleRNN(64),
        Dense(vocab_size, activation='softmax')
    ])
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    model.fit(X, y, epochs=300, verbose=0)

    # Generate
    generated = seed_notes.copy()
    for _ in range(sequence_length):
        encoded = [note_to_int[n] for n in generated[-max_len:]]
        padded = pad_sequences([encoded], maxlen=max_len, padding='pre')
        pred = model.predict(padded, verbose=0)
        next_index = np.argmax(pred)
        next_note = int_to_note[next_index]
        generated.append(next_note)

    print("Seed:", seed_notes)
    print("Generated Sequence:", generated)

# Call for multiple ragas
train_and_generate_raga("Bhairav", ['Sa', 'Reb'])
train_and_generate_raga("Bhopali", ['Sa', 'Re'])
train_and_generate_raga("Bageshree", ['Sa', 'Gab'])
train_and_generate_raga("Yaman", ['Sa', 'Re'])
train_and_generate_raga("Desh", ['Sa', 'Re'])


Training on Raga: Bhairav
Seed: ['Sa', 'Reb']
Generated Sequence: ['Sa', 'Reb', 'Ga', 'Ma', 'Pa', 'Dhab', 'Ni', 'Sha', 'Sha', 'Sha', 'Sha', 'Sha']

Training on Raga: Bhopali
Seed: ['Sa', 'Re']
Generated Sequence: ['Sa', 'Re', 'Ga', 'Pa', 'Dha', 'Sha', 'Sha', 'Dha', 'Dha', 'Sha', 'Dha', 'Dha']

Training on Raga: Bageshree
Seed: ['Sa', 'Gab']
Generated Sequence: ['Sa', 'Gab', 'Ma', 'Dha', 'Ni', 'Sha', 'Sha', 'Ni', 'Sha', 'Sha', 'Ni', 'Sha']

Training on Raga: Yaman
Seed: ['Sa', 'Re']
Generated Sequence: ['Sa', 'Re', 'Ga', 'Ma#', 'Pa', 'Dha', 'Ni', 'Sha', 'Dha', 'Ni', 'Pa', 'Dha']

Training on Raga: Desh
Seed: ['Sa', 'Re']
Generated Sequence: ['Sa', 'Re', 'Ma', 'Pa', 'Ni', 'Sha', 'Sha', 'Ni', 'Sha', 'Ni', 'Sha', 'Ni']
